<a href="https://colab.research.google.com/github/umangSharmacs/Charles_research_codes/blob/main/cnn_dailymail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.corpus import stopwords
import csv
from nltk.tag import pos_tag # for proper noun
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import math
from nltk.stem import PorterStemmer
import re
import pytrends
from pytrends.request import TrendReq
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tag import DefaultTagger
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from scipy import spatial

import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from bert_serving.client import BertClient

import gensim
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import remove_stopwords

from IPython.display import clear_output

In [ ]:
import os

corpus=[]
summary=[]
path="C:\\Users\\umang\\Desktop\\Research\\cnn\\stories"

for file in os.listdir(path):
    with open(f"{path}\\{file}", 'r', encoding='utf-8') as f:
        corpus.append(f.read())

articles=[]
summary=[]
for article in corpus:
    txt=[]
    for index,sentence in enumerate(article.split('\n')):
        if sentence=='@highlight':
            break
        if sentence!='':
            txt.append(sentence)
    s='. '.join(txt)
    if s!='':
        articles.append('. '.join(txt))
    else:
        continue
    summ=[]
    for s_index in range(index,len(article.split('\n'))):
        if article.split('\n')[s_index]!='@highlight' and article.split('\n')[s_index]!='':
            summ.append(article.split('\n')[s_index])
    summary.append('. '.join(summ))

In [ ]:
def preprocess(text):
    
    sent_tokens = nltk.sent_tokenize(text)
   
    word_tokens = nltk.word_tokenize(text)
    word_tokens_lower=[word.lower() for word in word_tokens]
    
    stopWords = list(set(stopwords.words("english")))
    word_tokens_refined=[x for x in word_tokens_lower if x not in stopWords]
    #print(len(word_tokens_refined))
    
    stem = []
    ps = PorterStemmer()
    for w in word_tokens_refined:
        stem.append(ps.stem(w))
    word_tokens_refined=stem
    
    return text,sent_tokens,word_tokens,word_tokens_lower,word_tokens_refined

In [ ]:
def frequency(word_tokens_refined, sent_tokens):
    ps = PorterStemmer()
    freqTable = {}
    for word in word_tokens_refined:    
        if word in freqTable:         
            freqTable[word] += 1    
        else:         
            freqTable[word] = 1
    for k in freqTable.keys():
        freqTable[k]= math.log10(1+freqTable[k])
    #Compute word frequnecy score of each sentence
    word_frequency={}
    for sentence in sent_tokens:
        word_frequency[sentence]=0
        e=nltk.word_tokenize(sentence)
        f=[]
        for word in e:
            f.append(ps.stem(word))
        for word,freq in freqTable.items():
            if word in f:
                word_frequency[sentence]+=freq
    maximum=max(word_frequency.values())
    for key in word_frequency.keys():
        try:
            word_frequency[key]=word_frequency[key]/maximum
            word_frequency[key]=round(word_frequency[key],3)
        except ZeroDivisionError:
            x=0
#    print(word_frequency.values())
    return freqTable, word_frequency

In [ ]:
def numerical_data(sent_tokens):
    numeric_data={}
    for sentence in sent_tokens:
        numeric_data[sentence] = 0
        word_tokens = nltk.word_tokenize(sentence)
        for k in word_tokens:
            if k.isdigit():
                numeric_data[sentence] += 1
    max_freq=max(numeric_data.values())
    for k in numeric_data.keys():
        try:
            numeric_data[k] = numeric_data[k]/max_freq
            numeric_data[k] = round(numeric_data[k], 3)
        except ZeroDivisionError:
            x=0
    return numeric_data

In [ ]:
def sentence_length(sent_tokens):
    sent_len_score={}
    for sentence in sent_tokens:
        sent_len_score[sentence] = 0
        word_tokens = nltk.word_tokenize(sentence)
        if len(word_tokens) in range(0,10):
            sent_len_score[sentence]=1-0.05*(10-len(word_tokens))
        elif len(word_tokens) in range(7,20):                    # calculate ideal sentence length using some other criteria? 
            sent_len_score[sentence]=1
        else:
            sent_len_score[sentence]=1-(0.05)*(len(word_tokens)-20)
    for k in sent_len_score.keys():
        sent_len_score[k]=round(sent_len_score[k],4)
    #print(sent_len_score.values())
    return sent_len_score

In [ ]:
def sentence_position(sent_tokens):
    sentence_position={}
    d=1
    no_of_sent=len(sent_tokens)
    for i in range(no_of_sent):
        a=1/d
        b=1/(no_of_sent-d+1)
        sentence_position[sent_tokens[d-1]]=max(a,b)
        d=d+1
    for k in sentence_position.keys():
        sentence_position[k]=round(sentence_position[k],3)
#    print(sentence_position.values())
    return sentence_position

In [ ]:
 def uppercase(sent_tokens):
    upper_case={}
    for sentence in sent_tokens:
        upper_case[sentence] = 0
        word_tokens = nltk.word_tokenize(sentence)
        for k in word_tokens:
            if k.isupper():
                upper_case[sentence] += 1
    maximum_frequency = max(upper_case.values())
    for k in upper_case.keys():
        try:
            upper_case[k] = (upper_case[k]/maximum_frequency)
            upper_case[k] = round(upper_case[k], 3)
        except ZeroDivisionError:
            x=0
#    print(upper_case.values())
    return upper_case

In [ ]:
def propernouns(sent_tokens):
    proper_noun={}
    for sentence in sent_tokens:
        tagged_sent = pos_tag(sentence.split())
        propernouns = [word for word, pos in tagged_sent if pos == 'NNP']
        proper_noun[sentence]=len(propernouns)
    maximum_frequency = max(proper_noun.values())
    for k in proper_noun.keys():
        try:
            proper_noun[k] = (proper_noun[k]/maximum_frequency)
            proper_noun[k] = round(proper_noun[k], 3)
        except ZeroDivisionError:
            x=0
#    print(proper_noun.values())
    return proper_noun

In [ ]:
def vectorize(word2vec, sentence):
    vec=0
    for word in sentence:
        if word in word2vec.wv.vocab:
            vec+= word2vec.wv[word]
    return vec

In [ ]:
def sentence_similarity(all_words,test_article,test_summ):
    # train model 
    word2vec = Word2Vec(all_words, min_count=2)
    
    summary_vectors=[]
    for sentence in test_summ:
        summary_vectors.append(vectorize(word2vec,sentence))

    label={}
    for sentence in test_article:
        label[sentence]=0
    
    for vector in summary_vectors:
        sim=[]
        for sentence in test_article:
            vec=0
            for word in sentence:
                if word in word2vec.wv.vocab:
                    vec+=word2vec.wv[word]
            sim.append(1 - spatial.distance.cosine(vec, vector))
        label[test_article[sim.index(max(sim))]]=1
        sim[sim.index(max(sim))]=0
        label[test_article[sim.index(max(sim))]]=1
    
#     for sentence in test_article:
#         if sentence in label:
#             continue
#         vec=0
#         label_check=False
#         for word in sentence:
#             if word in word2vec.wv.vocab:
#                 vec+=word2vec.wv[word]
#         for sumvec in summary_vectors:
#             similarity=1 - spatial.distance.cosine(vec, sumvec)
#             if similarity>0.93:
#                 label[sentence]=1
#                 label_check=True
#                 break
#         if label_check==False:
#             label[sentence]=0
    return label

In [ ]:
df_model=pd.DataFrame(columns=['Numerical_Data',
                               'Sentence_length',
                               'Sentence_position',
                               #'Word_frequency',
                               'Uppercase',
                               'Proper_nouns',
                               #'Headmatch',
                               #'Sentence_similarity',
                               #'TFIDF',
                               'Label',
                               'Key'])
print(df_model)

Empty DataFrame
Columns: [Numerical_Data, Sentence_length, Sentence_position, Uppercase, Proper_nouns, Label, Key]
Index: []


In [ ]:
for index,file in enumerate(articles[90000:]):
    clear_output(wait=True)
    print(index,".......................", end=" ")
    text=articles[index]
    text1=summary[index]
    sent_tokens = nltk.sent_tokenize(text)
    word_tokens = nltk.word_tokenize(text)
    sent_tokens1 = nltk.sent_tokenize(text1)
    word_tokens1 = nltk.word_tokenize(text1)
    word_tokens_lower=[word.lower() for word in word_tokens]
    stopWords = list(set(stopwords.words("english")))
    word_tokens_refined=[x for x in word_tokens_lower if x not in stopWords]

    numerical_data_dict=numerical_data(sent_tokens)

    sentence_length_dict=sentence_length(sent_tokens)
 
    sentence_position_dict=sentence_position(sent_tokens)

    freqTable, word_frequency_dict=frequency(word_tokens_refined, sent_tokens)

    uppercase_dict=uppercase(sent_tokens)

    propernouns_dict=propernouns(sent_tokens)

#    sent_similarity_dict=sentence_similarity(sent_tokens)

#    tfidf_score=tfidf(sent_tokens,index+1)
    
    Label=sentence_similarity(word_tokens_refined,sent_tokens, sent_tokens1)

    df_model=df_model.append(pd.DataFrame({
        'Numerical_Data':list(numerical_data_dict.values()),
        'Sentence_length': list(sentence_length_dict.values()),
        'Sentence_position':list(sentence_position_dict.values()),
        'Word_frequency':list(word_frequency_dict.values()),
        'Uppercase':list(uppercase_dict.values()),
        'Proper_nouns':list(propernouns_dict.values()),
        #'TFIDF':list(tfidf_score.values()),
        'Label':list(Label.values()),
        'Key':list(numerical_data_dict.keys())})
                              ,ignore_index=True)
    
    print(' : done')

2464 .......................  : done


In [ ]:
df_model['Label'].sum()

13874

In [ ]:
df_model.to_csv('cnn_10.csv')

In [ ]:
def compute_results(x_train, y_train,x_test, y_test):
    y_pred=[]
    models=[]
    # KNN
    for i in [3,5,10,15,20,50]:
        neigh = KNeighborsClassifier(n_neighbors=i)
        neigh.fit(x_train, y_train)
        y_pred.append(neigh.predict(x_test))
        models.append(f'KNN_{i}')
        print(f'KNN_{i} done')
        
    # Logistic Regression
    clf = LogisticRegression(random_state=0).fit(x_train, y_train)
    y_pred.append(clf.predict(x_test))
    models.append('Logistic_Regression')
    print('Logistic Regression done')
    
    # SVM
#     clf = svm.SVC()
#     clf.fit(x_train, y_train)
#     y_pred.append(clf.predict(x_test))
#     models.append('SVM')
#     print('SVM done')
    
    # Random Forest
    for i in range(10,110,10):
        classifier = RandomForestClassifier(n_estimators = i,criterion='entropy',random_state=0)
        classifier.fit(x_train, y_train)
        y_pred.append(classifier.predict(x_test))
        models.append(f'Random_Forest_{i}')
        print(f'Random_Forest_{i} done')
    acc=[]
    f1=[]
    cm=[]
    for index,i in enumerate(y_pred):
        cm.append(confusion_matrix(y_test, y_pred[index]))
        acc.append(metrics.accuracy_score(y_test, y_pred[index])*100)
        f1.append(metrics.f1_score(y_test, y_pred[index])*100)
    results=pd.DataFrame()
    results['Models']=models
    results['Accuracy']=acc
    results['F1 score']=f1
    results['Confusion Matrix']=cm
    return results

In [ ]:
df_model=pd.read_csv('cnn_2.csv')
df_model.head()

In [ ]:
x=df_model.drop(['Label', 'Key'], axis=1)
y=df_model.Label.astype('int')
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=0)
results=compute_results(x_train, y_train,x_test, y_test)

In [ ]:
results=results.sort_values('Accuracy', ascending=False)
results

In [ ]:
clf = svm.SVC()
clf.fit(x_train, y_train)
y_pred.append(clf.predict(x_test))
models.append('SVM')
print('SVM done')